# AWS setup

## **1. Launch an EC2 Instance**

### **Step 1: Open the EC2 Console**

1. Sign in to the [AWS Management Console](https://console.aws.amazon.com/ec2/).
2. In the navigation bar, select a region that is closest to you.

### **Step 2: Launch an Instance**

1. From the EC2 dashboard, click **Launch Instance**.
2. Under **Name and tags**, enter a name for your instance.
3. Under **Application and OS Images (Amazon Machine Image)**, select **Ubuntu**.
4. Choose an instance type, such as `t2.micro` (Free Tier eligible).
5. Under **Key pair (login)**, select **Create new key pair** or choose an existing one:
   - If creating a new key pair, download the `.pem` file and save it to `~/.ssh/` on your local machine.
   - Run `chmod 400 ~/.ssh/name.pem` to set the correct permissions.

### **Step 3: Configure Network Settings**

1. Under **Network settings**, ensure that a security group is created or selected.
2. Add a rule to allow SSH access:
   - Type: SSH
   - Protocol: TCP
   - Port Range: 22
   - Source: My IP (to restrict access to your IP address).

### **Step 4: Launch the Instance**

1. Review your settings and click **Launch Instance**.
2. Wait for the instance to be initialized.

## **2. Connect to Your EC2 Instance**

### **Step 1: Retrieve the Public IP Address**

1. Go to the **Instances** section in the EC2 console.
2. Select your instance and note the **Public IPv4 address**.

### **Step 2: Connect via SSH**

Use the following command to connect to your instance:

```bash
ssh -i ~/.ssh/name.pem ubuntu@your_public_ip
```

Replace `name.pem` with your key pair file name and `your_public_ip` with the instance's public IP address.

## **3. Install and Configure MariaDB**

### **Step 1: Update the System**

```bash
sudo apt update && sudo apt upgrade -y
```

### **Step 2: Install MariaDB**

```bash
sudo apt install mariadb-server -y
```

### **Step 3: Secure MariaDB Installation**

Run the security script to set up a root password and secure the installation:

```bash
sudo mysql_secure_installation
```

### **Step 4: Verify MariaDB Installation**

Check the status of the MariaDB service:

```bash
sudo systemctl status mariadb
```

Log in to the MariaDB shell:

```bash
sudo mysql -u root -p
```

## **4. Configure Security Groups for Database Access**

### **Step 1: Modify Security Group**

1. Go to the **Security Groups** section in the EC2 console.
2. Select the security group associated with your instance.
3. Add an inbound rule to allow MySQL access:
   - Type: MySQL/Aurora
   - Protocol: TCP
   - Port Range: 3306
   - Source: Anywhere (0.0.0.0/0) (Note: This is not recommended for production environments due to security risks).

### **Step 2: Configure MariaDB for Remote Access (Optional)**

If you need to allow remote access to MariaDB:

1. Edit the MariaDB configuration file:

```bash
sudo nano /etc/mysql/mariadb.conf.d/50-server.cnf
```

2. Find the line with `bind-address` and change it to:

```ini
bind-address = 0.0.0.0
```

3. Save and close the file, then restart MariaDB:

```bash
sudo systemctl restart mariadb
```

4. Grant remote access to the root user:

```sql
GRANT ALL PRIVILEGES ON *.* TO 'root'@'%' IDENTIFIED BY 'your_password' WITH GRANT OPTION;
FLUSH PRIVILEGES;
```

Replace `'your_password'` with the desired password for the root user.

## **5. Connect to MariaDB Remotely**

From your local machine, you can connect to the MariaDB server using:

```bash
mysql -u root -p -h your_public_ip
```

Replace `your_public_ip` with the EC2 instance's public IP address.

By following these steps, you will have set up an Ubuntu EC2 instance on AWS, configured it for SSH access, installed MariaDB, and set up security rules for remote database access.

# Updating kernel

1. `sudo apt update && sudo apt upgrade -y`
2. `sudo apt install linux-aws`
3. `sudo reboot`

# Mariadb server

## **1. Update and Upgrade the System**

First, ensure your system packages are up to date:

```bash
sudo apt update && sudo apt upgrade -y
```

## **2. Install MariaDB Server**

Next, install the MariaDB server package:

```bash
sudo apt install mariadb-server -y
```

## **3. Start and Enable MariaDB Service**

Start the MariaDB service and enable it to start on boot:

```bash
sudo systemctl start mariadb
sudo systemctl enable mariadb
```

## **4. Secure MariaDB Installation**

Run the security script to set up a root password and secure the installation:

```bash
sudo mysql_secure_installation
```

You will be prompted to:
- Set a root password
- Remove anonymous users
- Disallow root login remotely
- Remove test database and access to it
- Reload privilege tables

## **5. Verify MariaDB Installation**

Check the status of the MariaDB service to ensure it is running:

```bash
sudo systemctl status mariadb
```

You can also verify the installation by logging into the MariaDB shell:

```bash
sudo mysql -u root -p
```

## **6. Create a Database and User**

Once logged into the MariaDB shell, you can create a database and a user with the necessary privileges:

```sql
CREATE DATABASE example_db;
CREATE USER 'example_user'@'localhost' IDENTIFIED BY 'password';
GRANT ALL PRIVILEGES ON example_db.* TO 'example_user'@'localhost';
FLUSH PRIVILEGES;
EXIT;
```

## **7. Configure Remote Access (Optional)**

If you need to allow remote access to your MariaDB server, you will need to edit the MariaDB configuration file:

```bash
sudo nano /etc/mysql/mariadb.conf.d/50-server.cnf
```

Find the line that starts with `bind-address` and change its value to `0.0.0.0` to allow connections from any IP address:

```ini
bind-address = 0.0.0.0
```

Save and close the file, then restart the MariaDB service:

```bash
sudo systemctl restart mariadb
```

Additionally, configure the firewall to allow traffic on the MariaDB port (default is 3306):

```bash
sudo ufw allow 3306
```

## **8. Test Remote Connection**

You can test the remote connection using a MariaDB client from another machine:

```bash
mysql -u example_user -p -h your_server_ip -D example_db
```
